![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

**1. Reading in and Splitting the data **

In [25]:
# Import libraries
import pandas as pd
import numpy as np

As only a csv file "bank_marketing.csv" is supplied, the first step is to read in and examine the data and its features:

In [26]:
# Read in the data file
data = pd.read_csv("bank_marketing.csv")
data.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


It is clear that the given data files contain all data features in one file which is not inappropriate for the bank's operation as each combination of data features will serve different purposes. Therefore, splitting data into different subsets is necessary based on the given contents above:

In [27]:
# Subset the client data 
client = data[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [28]:
# Subset the campaign data 
campaign = data[["client_id", "number_contacts", "month", "day", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
campaign.head()

,client_id,number_contacts,month,day,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,0,nonexistent,no
1,1,1,may,19,149,0,nonexistent,no
2,2,1,may,23,226,0,nonexistent,no
3,3,1,may,27,151,0,nonexistent,no
4,4,1,may,3,307,0,nonexistent,no


In [29]:
# Subset the economics data 
economics = data[["client_id", "cons_price_idx", "euribor_three_months"]]
economics.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


As the project context is on personal loan, all subsets have one column `client_id` in common. Now that subsets are splitted successfully, we can process the cleaning step. 

**2. Cleaning the data **

Firstly, we examine the data type and null values of all features to identify any errors before applying the cleaning techniques:

In [30]:
# Examine the data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

Overall, only 4 data features `campaign_outcome`, `previous_outcome`, `credit_default` and `mortgage` should be changed to boolean values which are more suitable for future modelling tasks. No null values are detected, we will only need to identify their categorical values before converting their data types:

In [31]:
# Convert columns to bool data type
for col in ["credit_default", "mortgage"]:
    client[col] = client[col].astype(bool)

In [32]:
# Values of campaign_outcome column
campaign["campaign_outcome"].value_counts()

no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [33]:
# Values of previous_outcome column
campaign["previous_outcome"].value_counts()

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

To boolean `campaign_outcome` and `previous_outcome` values, we will change from "yes"/"success" to 1, and from "no"/"failure"/"nonexistent" to 0. The data type will also be converted to float after changing their values.

In [34]:
# Editing the campaign dataset
# Change campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Convert poutcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0,
                                                                 "nonexistent": 0})

In [35]:
# Convert data type
for col in ["campaign_outcome", "previous_outcome"]:
    campaign[col] = campaign[col].astype(bool)

Besides, 2 text columns `education` and `job` have one inappropriate character `.`. Therefore, it needs to be removed or changed to another formatting way:

In [36]:
## Editing the client dataset
# Clean education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

In [37]:
# Clean job column
client["job"] = client["job"].str.replace(".", "")

Lastly, new date column will also be created with a combination of year, month, day:

In [38]:
# Capitalize month and day columns
campaign["month"] = campaign["month"].str.capitalize()

# Add year column
campaign["year"] = "2022"

# Convert day to string
campaign["day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

After the proper `last_contact_date` is successfully created, the component columns can be now removed:

In [39]:
# Drop unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

**3. Saving the data **

Last step is to Save the three DataFrames to csv files, without an index, as `client.csv`, `campaign.csv`, and `economics.csv` respectively.

In [40]:
# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)